## Roll Rate Analysis

En esta sección, se utilizará el la metodología *Roll Rate Analysis* para determinar de una manera más acorde a los datos, aquellos clientes que se pueden identificar como buenos o malos, dependiendo de los días de atraso se registran en el pago de sus cuentas.

In [1]:
import pandas as pd 
import numpy as np 

# Se actualiza el formato de números flotantes
pd.options.display.float_format = '{:,.2f}'.format

## 1. Carga de datos

In [2]:
credit = pd.read_csv('../data/credit_record.csv')
credit

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [3]:
# Validamos la cantidad de ID (clientes) únicos
credit['ID'].nunique()

45985

## 2. Estructuración de dataset

Se recategorizarán los valores `X` y `C` de la variable *STATUS*, con el fin de mantener únicamente valores numéricos

In [4]:
credit['STATUS'] = credit['STATUS'].replace({'X':-2, 'C':-1})
credit['STATUS'] = credit['STATUS'].astype('int')
credit.dtypes

ID                int64
MONTHS_BALANCE    int64
STATUS            int64
dtype: object

In [5]:
# Validamos los valores de la variable STATUS
credit['STATUS'].value_counts()

-1    442031
 0    383120
-2    209230
 1     11090
 5      1693
 2       868
 3       320
 4       223
Name: STATUS, dtype: int64

In [6]:
# Validamos los valores de la variable MONTHS_BALANCE
credit['MONTHS_BALANCE'].value_counts()

-1     34436
-2     34209
 0     33856
-3     33854
-4     33365
       ...  
-56     2109
-57     1649
-58     1255
-59      820
-60      415
Name: MONTHS_BALANCE, Length: 61, dtype: int64

## 3. Roll Rate analysis


In [7]:
# Se pivotea el df para procesar los datos de una manera correcta
pivot_tb = pd.pivot(credit, index='ID', columns='MONTHS_BALANCE', values='STATUS').reset_index()
pivot_tb.head(10)

MONTHS_BALANCE,ID,-60,-59,-58,-57,-56,-55,-54,-53,-52,...,-9,-8,-7,-6,-5,-4,-3,-2,-1,0
0,5001711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,-2.00
1,5001712,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00
2,5001713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00
3,5001714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00
4,5001715,NaN,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,...,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00
5,5001717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,0.00,0.00,0.00,-1.00,-1.00,-1.00,-1.00,-1.00
6,5001718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.00,1.00,0.00,0.00,0.00,0.00,0.00,-1.00,-1.00,-1.00
7,5001719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00
8,5001720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.00,0.00,1.00,1.00,0.00,1.00,1.00,0.00,0.00,1.00
9,5001723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00,-2.00


### 3.1. Análisis 1 a 12 meses

En primer lugar se realiza el análisis tomando en cuenta los valores registrados en el mes actual vs 12 meses en libros


In [8]:
pivot_tb_current_month = pivot_tb[['ID', 0, -11]]
pivot_tb_current_month = pivot_tb_current_month[(pivot_tb_current_month[0].notna()) & (pivot_tb_current_month[-11].notna())]
pivot_tb_current_month = pivot_tb_current_month[(pivot_tb_current_month[0] != -2) & (pivot_tb_current_month[-11] != -2)]
pivot_tb_current_month = pivot_tb_current_month.rename(columns={'ID':'id',0:'current_month', -11:'one_year'})

pivot_tb_current_month['current_month'] = pivot_tb_current_month['current_month'].astype('int')
pivot_tb_current_month['one_year'] = pivot_tb_current_month['one_year'].astype('int')
pivot_tb_current_month

# Generamos la matriz de Transición
matriz_current_month = np.zeros((7,7),float)

for i in range(-1, 6):

    for j in range(-1, 6):
        matriz_current_month[i,j] = pivot_tb_current_month.loc[(pivot_tb_current_month['one_year']==i) & (pivot_tb_current_month['current_month']==j),'one_year'].count()
        matriz_current_month[i,j] = (matriz_current_month[i,j]*100)/ pivot_tb_current_month.loc[pivot_tb_current_month['one_year']==i,'one_year'].count()
    

matriz_current_month_df = pd.DataFrame(matriz_current_month, columns=[-1,0,1,2,3,4,5], index=[-1,0,1,2,3,4,5])
matriz_current_month_df

,-1,0,1,2,3,4,5
-1,47.35,1.42,0.06,0.01,0.03,0.33,50.81
0,35.86,8.08,0.00,0.00,0.00,3.03,53.03
1,38.46,0.00,0.00,7.69,0.00,7.69,46.15
2,0.00,0.00,0.00,0.00,0.00,0.00,100.00
3,25.00,0.00,0.00,0.00,0.00,75.00,0.00
4,0.00,2.33,0.00,0.00,0.00,48.84,48.84
5,0.00,0.00,0.00,0.00,0.00,0.00,100.00


### 3.2. Análisis 12 meses a 18 meses

In [9]:
pivot_tb_one_year = pivot_tb[['ID', -11, -17]]
pivot_tb_one_year = pivot_tb_one_year[(pivot_tb_one_year[-11].notna()) & (pivot_tb_one_year[-17].notna())]
pivot_tb_one_year = pivot_tb_one_year[(pivot_tb_one_year[-11] != -2) & (pivot_tb_one_year[-17] != -2)]
pivot_tb_one_year = pivot_tb_one_year.rename(columns={'ID':'id',-11:'twelve_months', -17:'eighteen_months'})

pivot_tb_one_year['twelve_months'] = pivot_tb_one_year['twelve_months'].astype('int')
pivot_tb_one_year['eighteen_months'] = pivot_tb_one_year['eighteen_months'].astype('int')

# Generamos la matriz de Transición
matriz_one_year = np.zeros((7,7),float)

for i in range(-1, 6):

    for j in range(-1, 6):
        matriz_one_year[i,j] = pivot_tb_one_year.loc[(pivot_tb_one_year['eighteen_months']==i) & (pivot_tb_one_year['twelve_months']==j),'eighteen_months'].count()
        matriz_one_year[i,j] = (matriz_one_year[i,j]*100)/ pivot_tb_one_year.loc[pivot_tb_one_year['eighteen_months']==i,'eighteen_months'].count()
    

matriz_one_year_df = pd.DataFrame(matriz_one_year, columns=[-1,0,1,2,3,4,5], index=[-1,0,1,2,3,4,5])
matriz_one_year_df


,-1,0,1,2,3,4,5
-1,66.27,2.25,0.10,0.03,0.03,0.11,31.22
0,44.68,12.77,1.06,0.00,0.53,2.13,38.83
1,33.33,0.00,16.67,0.00,0.00,8.33,41.67
2,27.27,0.00,0.00,0.00,0.00,36.36,36.36
3,0.00,0.00,0.00,0.00,0.00,40.00,60.00
4,0.00,0.00,0.00,0.00,0.00,85.29,14.71
5,0.00,0.00,0.00,0.00,0.00,0.00,100.00


### 3.3. Análisis de 24 a 36 meses en libros

In [10]:
pivot_tb_three_years = pivot_tb[['ID', -23, -35]]
pivot_tb_three_years = pivot_tb_three_years[(pivot_tb_three_years[-23].notna()) & (pivot_tb_three_years[-35].notna())]
pivot_tb_three_years = pivot_tb_three_years[(pivot_tb_three_years[-23] != -2) & (pivot_tb_three_years[-35] != -2)]
pivot_tb_three_years = pivot_tb_three_years.rename(columns={'ID':'id',-23:'two_years', -35:'three_years'})

pivot_tb_three_years['two_years'] = pivot_tb_three_years['two_years'].astype('int')
pivot_tb_three_years['three_years'] = pivot_tb_three_years['three_years'].astype('int')

# Matriz de Transición
matriz_three_years = np.zeros((7,7),float)

for i in range(-1, 6):

    for j in range(-1, 6):
        matriz_three_years[i,j] = pivot_tb_three_years.loc[(pivot_tb_three_years['two_years']==i) & (pivot_tb_three_years['three_years']==j),'two_years'].count()
        matriz_three_years[i,j] = (matriz_three_years[i,j]*100)/ pivot_tb_three_years.loc[pivot_tb_three_years['two_years']==i,'two_years'].count()
    

matriz_three_years_df = pd.DataFrame(matriz_three_years, columns=[-1,0,1,2,3,4,5], index=[-1,0,1,2,3,4,5])
matriz_three_years_df

,-1,0,1,2,3,4,5
-1,97.82,2.03,0.05,0.00,0.10,0.00,0.00
0,82.54,17.46,0.00,0.00,0.00,0.00,0.00
1,100.00,0.00,0.00,0.00,0.00,0.00,0.00
2,75.00,0.00,0.00,25.00,0.00,0.00,0.00
3,100.00,0.00,0.00,0.00,0.00,0.00,0.00
4,40.00,8.00,4.00,0.00,0.00,48.00,0.00
5,39.71,1.06,0.08,0.04,0.07,0.11,58.92


### 3.4. Análisis 36 a 48 meses en libros

In [11]:
# 36 meses a 48 meses
pivot_tb_four_years = pivot_tb[['ID', -35, -47]]
pivot_tb_four_years = pivot_tb_four_years[(pivot_tb_four_years[-35].notna()) & (pivot_tb_four_years[-47].notna())]
pivot_tb_four_years = pivot_tb_four_years[(pivot_tb_four_years[-35] != -2) & (pivot_tb_four_years[-47] != -2)]
pivot_tb_four_years = pivot_tb_four_years.rename(columns={'ID':'id',-35:'two_years', -47:'three_years'})

pivot_tb_four_years['two_years'] = pivot_tb_four_years['two_years'].astype('int')
pivot_tb_four_years['three_years'] = pivot_tb_four_years['three_years'].astype('int')


# Matriz de Transición
matriz_four_years = np.zeros((7,7),float)

for i in range(-1, 6):

    for j in range(-1, 6):
        matriz_four_years[i,j] = pivot_tb_four_years.loc[(pivot_tb_four_years['two_years']==i) & (pivot_tb_four_years['three_years']==j),'two_years'].count()
        matriz_four_years[i,j] = (matriz_four_years[i,j]*100)/ pivot_tb_four_years.loc[pivot_tb_four_years['two_years']==i,'two_years'].count()
    

matriz_twofour_years_df = pd.DataFrame(matriz_four_years, columns=[-1,0,1,2,3,4,5], index=[-1,0,1,2,3,4,5])
matriz_twofour_years_df

,-1,0,1,2,3,4,5
-1,97.51,2.14,0.27,0.00,0.00,0.09,0.00
0,82.35,15.69,0.00,1.96,0.00,0.00,0.00
1,100.00,0.00,0.00,0.00,0.00,0.00,0.00
2,33.33,66.67,0.00,0.00,0.00,0.00,0.00
3,100.00,0.00,0.00,0.00,0.00,0.00,0.00
4,60.00,5.00,5.00,0.00,10.00,20.00,0.00
5,62.02,1.62,0.23,0.19,0.03,0.10,35.81


## Conclusión

Se realizó el `roll rate analysis` de la información disponible para varios periodos de tiempo y aunque no se puede observar una tendencia fija con que se pueda determinar un nivel en el cuál un cliente pueda condiderarse un *mal cliente*, se observa realizando un `Roll Backward` que aquellos cliente con un nivel 1 (30 a 59 días de atraso) tienden a no pasar a niveles de morosidad más bajos.

Por tanto, para la etiqueta o valor a predecir los valores con un valor menor o igual a 1 se considerarán malos clientes o clientes que pueden caer en morosidad y en cambio el resto de valores se considerarán como buenos clientes.